In [2]:
import json
import pandas as pd
from glob import glob

In [33]:
videos = [x.split('/')[-1].split('.')[0] for x in glob('annotations/**/*.json', recursive=True)]
df = pd.DataFrame(videos).rename(columns={0:'video'})

In [38]:
df.to_excel('single_ape_subset.xlsx')

In [52]:
def modify_annotation(annotation_prefix, outpath_prefix, video_id, xmin_scaler=0, ymin_scaler=0, xmax_scaler=0, ymax_scaler=0):
    
    x_min_lim = 0
    y_min_lim = 0
    
    x_max_lim = 720
    y_max_lim = 404
    
    with open(f'{annotation_prefix}/{video_id}.json', 'rb') as handle:
        a = json.load(handle)
    
    for frame in a['annotations']:
        for d in frame['detections']:
            
            xmin, ymin, xmax, ymax = d['bbox'][0], d['bbox'][1], d['bbox'][2], d['bbox'][3]
            
            xmin -= xmin_scaler * (xmax - xmin)
            xmax += xmax_scaler * (xmax - xmin)
            ymin -= ymin_scaler * (ymax - ymin)
            ymax += ymax_scaler * (ymax - ymin)

            xmin = x_min_lim if (xmin < x_min_lim) else xmin
            ymin = y_min_lim if (ymin < y_min_lim) else ymin
            xmax = x_max_lim if (xmax > x_max_lim) else xmax
            ymax = y_max_lim if (ymax > y_max_lim) else ymax
            
            d['bbox'][0], d['bbox'][1], d['bbox'][2], d['bbox'][3] = xmin, ymin, 720, 404
            
    with open(f'{outpath_prefix}/{video_id}.json', 'w') as handle:
        json.dump(a, handle, ensure_ascii=False, indent=4)